# Diabetes Risk Prediction
This project uses the Behavioral Risk Factor Surveillance System (BRFSS) survey data from [this link](https://www.cdc.gov/brfss/annual_data/annual_2024.html) to predict the probability of developing different types of Diabetes. Features about U.S. residents include demographic data (e.g. income level, education, race) as well as data regarding health-related risk behaviors, chronic health conditions, and use of preventive services.

This is the first notebook for the project, which parses the raw ASCII data file, available in the link above, to extract the relevant target and feature variables for subsequent EDA and modeling in another notebook in this folder.

The dataset contains 2 identifier columns, 3 Target variable candidates and a total of 24 potential features, as described below:
- Each row is defined by a uniquely defined by (i.e. Table's Grain)
  1. "State FIPS Code"
  2. "Annual Sequence Number" 
- Target variable candidates related to Diabetes:
  1. "(Ever told) you had diabetes"
  2. "Ever been told by a doctor or other health professional that you have pre-diabetes or borderline diabetes?",
  3. "What type of diabetes do you have?"
- Demographic features:
  1. "Urban/Rural Status"
  2. "Reported age in five-year age categories calculated variable"
  3. "Sex of Respondent"
  4. "Computed Race-Ethnicity grouping"
  5. "Education Level"
  6. "Income Level"
- Personal health features:
  1. "Have Personal Health Care Provider?"
  2. "Could Not Afford To See Doctor"
  3. "Computed Weight in Kilograms"
  4. "Computed Height in Meters"
  5. "Computed body mass index"
  6. "Exercise in Past 30 Days"
  7. "How often did you drink regular soda or pop that contains sugar?"
  8. "How often did you drink sugar-sweetened drinks?"
  9. "Computed Smoking Status"
  10. "Computed number of drinks of alcohol beverages per week"
  11. "General Health"
- Other disease indicator features:
  1. "Ever Diagnosed with Heart Attack"
  2. "Ever Diagnosed with Angina or Coronary Heart Disease"
  3. "Ever Diagnosed with a Stroke"
  4. "Ever told you have kidney disease?"
  5. "Ever Told Had Asthma"
  6. "(Ever told) you had a depressive disorder"
  7. "Told Had Arthritis"

## Setup
### Define parameters

In [17]:
# URLs with input data
raw_data_url = "https://www.cdc.gov/brfss/annual_data/2024/files/LLCP2024ASC.zip"
data_dict_url = "https://www.cdc.gov/brfss/annual_data/2024/zip/codebook24_llcp-v2-508.zip"

# Define columns to extract using labels from HTML file
columns_to_extract = [
    "State FIPS Code",
    "Annual Sequence Number",
    "(Ever told) you had diabetes",
    "Ever been told by a doctor or other health professional that you have pre-diabetes or borderline diabetes?",
    "What type of diabetes do you have?",
    "Urban/Rural Status",
    "Reported age in five-year age categories calculated variable",
    "Sex of Respondent",
    "Computed Race-Ethnicity grouping",
    "Education Level",
    "Income Level",
    "Have Personal Health Care Provider?",
    "Could Not Afford To See Doctor",
    "Computed Weight in Kilograms",
    "Computed Height in Meters",
    "Computed body mass index",
    "Exercise in Past 30 Days",
    "How often did you drink regular soda or pop that contains sugar?",
    "How often did you drink sugar-sweetened drinks?",
    "Computed Smoking Status",
    "Computed number of drinks of alcohol beverages per week",
    "General Health",
    "Ever Diagnosed with Heart Attack",
    "Ever Diagnosed with Angina or Coronary Heart Disease",
    "Ever Diagnosed with a Stroke",
    "Ever told you have kidney disease?",
    "Ever Told Had Asthma",
    "(Ever told) you had a depressive disorder",
    "Told Had Arthritis"
]

# Output file for writing final dataframe
output_file = "diabetes_data.pickle"

### Import packages

In [18]:
from bs4 import BeautifulSoup
import pandas as pd
import pickle
import re

pd.set_option('display.max_columns', None)

### Define Functions

In [19]:
def parse_brfss_dictionary(html_file):
    """
    Parse HTML data dictionary to extract both column definitions and value-to-label mappings
    in a single pass through the file.
    
    Parameters:
    -----------
    html_file : str
        Path to the HTML data dictionary file
    
    Returns:
    --------
    tuple : (column_lookup, codebook)
        - column_lookup: dict mapping variable labels to metadata
          Format: {label: {'column_range': str, 'type': str, 'sas_name': str}}
        - codebook: dict mapping SAS variable names to value-label mappings
          Format: {sas_variable_name: {value: label}}
    """
    with open(html_file, 'r', encoding='windows-1252') as f:
        soup = BeautifulSoup(f, 'html.parser')
    
    column_lookup = {}
    codebook = {}
    
    # Find all variable tables (one pass through HTML)
    tables = soup.find_all('table', {'class': 'table'})
    
    for table in tables:
        # Extract metadata from header cell
        metadata_cell = table.find('td', {'class': 'l m linecontent'})
        if not metadata_cell:
            continue
        
        metadata_text = metadata_cell.get_text()
        
        # Only process cells that contain variable definitions
        if 'Label:' not in metadata_text or 'Column:' not in metadata_text:
            continue
        
        # Extract label (between "Label:" and "Section Name:")
        # Note: HTML uses \xa0 (non-breaking spaces)
        label_match = re.search(r'Label:[\s\xa0]+(.+?)Section[\s\xa0]+Name:', metadata_text)
        
        # Extract column range (format: "N" or "N-M")
        column_match = re.search(r'Column:[\s\xa0]+(\d+(?:-\d+)?)', metadata_text)
        
        # Extract variable type (Num or Char)
        type_match = re.search(r'Type[\s\xa0]+of[\s\xa0]+Variable:[\s\xa0]+(Num|Char)', metadata_text)
        
        # Extract SAS variable name (stops before "Question")
        varname_match = re.search(r'SAS[\s\xa0]+Variable[\s\xa0]+Name:[\s\xa0]+(\w+?)(?=Question)', metadata_text)
        
        if not (label_match and column_match and varname_match):
            continue
        
        # Store column metadata
        label = label_match.group(1).strip().replace('\xa0', ' ')
        column_range = column_match.group(1)
        var_type = type_match.group(1) if type_match else None
        var_name = varname_match.group(1)
        
        column_lookup[label] = {
            'column_range': column_range,
            'type': var_type,
            'sas_name': var_name
        }
        
        # Calculate column width for this variable
        # This determines the zero-padding needed for values
        if '-' in column_range:
            start, end = map(int, column_range.split('-'))
            column_width = end - start + 1
        else:
            column_width = 1
        
        # Extract value-label mappings from table body
        tbody = table.find('tbody')
        if not tbody:
            continue
        
        value_labels = {}
        has_categorical_values = False
        has_range_values = False  # Track if this variable has any range values
        
        for row in tbody.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) < 2:
                continue
            
            # Extract value (first column)
            value = cells[0].get_text(strip=True)
            
            # Check if this is a range value (e.g., "1 - 97", "100 - 999")
            # Range values indicate continuous/numeric variables
            if ' - ' in value:
                has_range_values = True
                continue  # Skip adding ranges to the codebook
            
            # Extract label (second column)
            label_html = cells[1]
            
            # Get text and split by line breaks to separate notes
            label_text = label_html.get_text(separator='|')
            label_parts = label_text.split('|')
            
            # Take first part (before notes)
            value_label = label_parts[0].strip()
            
            # Clean skip logic (remove "→Go to..." instructions)
            value_label = re.sub(r'→Go to.*$', '', value_label).strip()
            
            # Clean encoding issues: Replace "Donï¿½t know" and similar patterns with "Unknown"
            # This handles character encoding issues from the HTML
            if re.match(r'^Don.{1,3}t know', value_label, re.IGNORECASE):
                value_label = 'Unknown'
            # Also catch "Refused" variations and other missing data indicators
            elif value_label.lower().startswith('refused'):
                value_label = 'Refused'
            
            # Remove encoding error characters and everything after them (e.g., "Noï¿½Go" → "No")
            # The pattern ï¿½ is a common Unicode replacement character for malformed text
            value_label = re.sub(r'ï¿½.*$', '', value_label).strip()
            
            # Check if this is a categorical value (not just special codes)
            # Special codes: HIDDEN, BLANK, and codes like 7/9/77/99/777/999
            is_special = value in ['HIDDEN', 'BLANK'] or re.match(r'^[79]+$', value)
            if not is_special:
                has_categorical_values = True
            
            # Pad numeric values to match the column width in the ASCII file
            # This ensures codebook values match the fixed-width format
            if value.isdigit() and len(value) < column_width:
                value_padded = value.zfill(column_width)
            else:
                value_padded = value
            
            # Store mapping with properly padded value
            value_labels[value_padded] = value_label
        
        # Only add to codebook if:
        # 1. Has meaningful categorical values (not just HIDDEN/BLANK/special codes)
        # 2. Does NOT have any range values (which indicate continuous variables)
        # Variables with ranges (like _DRNKWK3, SSBSUGR2) should remain numeric
        if value_labels and has_categorical_values and not has_range_values:
            codebook[var_name] = value_labels
    
    print(f"Successfully parsed {len(column_lookup)} variable definitions and {len(codebook)} value label mappings from HTML dictionary")
    return column_lookup, codebook


def apply_value_labels(df, codebook, columns_to_label=None):
    """
    Apply value-to-label mappings to DataFrame columns.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with numeric/coded values
    codebook : dict
        Value label mappings from parse_brfss_dictionary()
    columns_to_label : list, optional
        Specific columns to label. If None, attempts to label all columns.
    
    Returns:
    --------
    pd.DataFrame : DataFrame with values replaced by labels
    """
    df_labeled = df.copy()
    
    if columns_to_label is None:
        columns_to_label = df.columns
    
    labeled_count = 0
    skipped_vars = []
    
    for col in columns_to_label:
        # Check if column has value labels in codebook
        if col not in codebook:
            continue
        
        value_map = codebook[col]
        
        # Test mapping on a sample to see if it's appropriate
        # If less than 50% of non-null values can be mapped, skip this variable
        # (it's likely a continuous variable or identifier)
        sample = df[col].dropna().head(1000)
        if len(sample) > 0:
            test_mapped = sample.astype(str).str.strip().map(value_map)
            mapping_success_rate = test_mapped.notna().sum() / len(sample)
            
            if mapping_success_rate < 0.5:
                skipped_vars.append(f"{col} ({mapping_success_rate:.1%} mappable)")
                continue
        
        # Apply mapping
        df_labeled[col] = df[col].astype(str).str.strip().map(value_map)
        
        # Count how many values were successfully mapped
        mapped = df_labeled[col].notna().sum()
        if mapped > 0:
            labeled_count += 1
            print(f"  Labeled {col}: {mapped:,} / {df[col].notna().sum():,} values ({mapped/df[col].notna().sum()*100:.1f}%)")
    
    if skipped_vars:
        print(f"\n  Skipped (continuous/identifier): {', '.join(skipped_vars)}")
    
    print(f"\nSuccessfully labeled {labeled_count} columns")
    return df_labeled

def parse_data_file(html_file, asc_file):
    # Step 1: Parse HTML data dictionary (single pass for both metadata and value labels)
    print("Parsing HTML data dictionary...")
    column_lookup, codebook = parse_brfss_dictionary(html_file)
    
    # Step 2: Convert labels to colspecs for pd.read_fwf()
    colspecs = []
    column_names = []
    dtypes = {}
    
    print("\nMapping columns:")
    for label in columns_to_extract:
        if label in column_lookup:
            col_info = column_lookup[label]
            col_range = col_info['column_range']
            
            # Parse "1-2" or "149" format
            if '-' in col_range:
                start, end = map(int, col_range.split('-'))
            else:
                start = end = int(col_range)
            
            # Convert to 0-based indexing for Python
            colspecs.append((start - 1, end))
            
            # Use SAS variable name for column name (to match codebook keys)
            col_name = col_info['sas_name']
            column_names.append(col_name)
            
            # Set dtype (start with string for safety, can convert later)
            dtypes[col_name] = str
            
            print(f"  {label} -> {col_name} (columns {col_range})")
        else:
            print(f"  WARNING: '{label}' not found in data dictionary")
    
    print(f"\nPrepared to extract {len(colspecs)} columns from ASCII file")
    
    # Step 3: Read the ASCII file using pd.read_fwf()
    print(f"\nReading ASCII file: {asc_file}")
    df = pd.read_fwf(
        asc_file, 
        colspecs=colspecs,
        names=column_names,
        dtype=dtypes,
        encoding='ascii'
    )
    
    print(f"Successfully loaded {len(df):,} rows and {len(df.columns)} columns")
    
    # Step 4: Apply value labels to DataFrame
    print("\nApplying value labels to DataFrame...")
    df = apply_value_labels(df, codebook)
    return df

## Download data
### Raw data file (ASCII)

In [20]:
asc_zip = !ls LLCP2024ASC*.zip
# If ascii zip file not present, download it and get zip file name
if not asc_zip:
    !wget {raw_data_url}
    asc_zip = !ls LLCP2024ASC*.zip

asc_file = !ls LLCP2024*.ASC*
# If raw ascii file not present, unzip ascii zip file and get raw ascii file name
if not asc_file:
    !unzip {asc_zip[0]}
    asc_file = !ls LLCP2024*.ASC*
asc_file = asc_file[0]

### Data dictionary file (HTML)

In [21]:
dict_zip = !ls codebook24_llcp*.zip
# If html zip file not present, download it and get html zip file name
if not dict_zip:
    !wget {data_dict_url}
    dict_zip = !ls codebook24_llcp*.zip

html_file = !ls USCODE24_LLCP*.HTML
# If raw html file not present, unzip html zip file and get raw html file name
if not html_file:
    !unzip {dict_zip[0]}
    html_file = !ls USCODE24_LLCP*.HTML
html_file = html_file[0]

## Parse data

In [22]:
data_df = parse_data_file(html_file, asc_file)

Parsing HTML data dictionary...
Successfully parsed 291 variable definitions and 247 value label mappings from HTML dictionary

Mapping columns:
  State FIPS Code -> _STATE (columns 1-2)
  Annual Sequence Number -> SEQNO (columns 36-45)
  (Ever told) you had diabetes -> DIABETE4 (columns 149)
  Ever been told by a doctor or other health professional that you have pre-diabetes or borderline diabetes? -> PREDIAB2 (columns 285)
  What type of diabetes do you have? -> DIABTYPE (columns 286)
  Urban/Rural Status -> _URBSTAT (columns 1403)
  Reported age in five-year age categories calculated variable -> _AGEG5YR (columns 1977-1978)
  Sex of Respondent -> SEXVAR (columns 88)
  Computed Race-Ethnicity grouping -> _RACE (columns 1972)
  Education Level -> EDUCA (columns 187)
  Income Level -> INCOME3 (columns 204-205)
  Have Personal Health Care Provider? -> PERSDOC3 (columns 110)
  Could Not Afford To See Doctor -> MEDCOST1 (columns 111)
  Computed Weight in Kilograms -> WTKG3 (columns 1989-1

## Transform
Apply transformations to convert raw values to their proper formats with correct decimal places and units.

### Apply Decimal Places to Numeric Variables
Convert numeric variables with implied decimal places according to the data dictionary specifications.

In [23]:
# Create a copy of the parsed dataframe for transformations
# This allows the Transform section to be rerun without corrupting data
transform_df = data_df.copy()

print(f"Created transform_df with {len(transform_df):,} rows and {len(transform_df.columns)} columns")

Created transform_df with 457,670 rows and 29 columns


In [24]:
# Define columns with implied decimal places (all have 2 decimal places per data dictionary)
decimal_transforms = {
    'WTKG3': {
        'name': 'Weight (kg)',
        'decimal_places': 2,
        'description': 'Computed Weight in Kilograms'
    },
    'HTM4': {
        'name': 'Height (m)',
        'decimal_places': 2,
        'description': 'Computed Height in Meters'
    },
    '_BMI5': {
        'name': 'BMI',
        'decimal_places': 2,
        'description': 'Computed Body Mass Index'
    }
}

print("Applying decimal place transformations:\n")

for col, config in decimal_transforms.items():
    if col in transform_df.columns:
        # Convert to numeric, coercing non-numeric values to NaN
        numeric_vals = pd.to_numeric(transform_df[col], errors='coerce')
        
        # Apply decimal transformation (divide by 10^decimal_places)
        divisor = 10 ** config['decimal_places']
        transform_df[col] = numeric_vals / divisor
        
        # Report transformation
        non_null = transform_df[col].notna().sum()
        print(f"{config['description']} ({col}):")
        print(f"  Transformed {non_null:,} values (÷{divisor})")
        print()

print("✓ Decimal transformations complete")

Applying decimal place transformations:

Computed Weight in Kilograms (WTKG3):
  Transformed 421,278 values (÷100)

Computed Height in Meters (HTM4):
  Transformed 433,610 values (÷100)

Computed Body Mass Index (_BMI5):
  Transformed 414,633 values (÷100)

✓ Decimal transformations complete


## Validation
### Grain Check
Confirm that the `_STATE` and `SEQNO` columns uniquely describe each row of the dataset.

In [25]:
print(f"Every row is uniquely defined by the `_STATE` and `SEQNO` columns: {len(transform_df) == len(transform_df.drop_duplicates(subset=['_STATE', 'SEQNO']))}")

Every row is uniquely defined by the `_STATE` and `SEQNO` columns: True


In [26]:
# Display summary of dataset
print("\nFirst 5 rows:")
display(transform_df.head())

print("\nDataset info:")
print(transform_df.info())

print("\nColumn value counts:")
for col in transform_df.columns:
    print(transform_df[col].value_counts().head(10))


First 5 rows:


,_STATE,SEQNO,DIABETE4,PREDIAB2,DIABTYPE,_URBSTAT,_AGEG5YR,SEXVAR,_RACE,EDUCA,INCOME3,PERSDOC3,MEDCOST1,WTKG3,HTM4,_BMI5,EXERANY2,SSBSUGR2,SSBFRUT3,_SMOKER3,_DRNKWK3,GENHLTH,CVDINFR4,CVDCRHD4,CVDSTRK3,CHCKDNY2,ASTHMA3,ADDEPEV3,HAVARTH4
0,Alabama,2024000001,No,NaN,NaN,"Urban counties (_URBNRRL = 1,2,3,4,5)",Age 75 to 79,Female,"White only, non-Hispanic",Grade 12 or GED (High school graduate),Refused,More than one,No,59.42,1.63,22.49,Yes,NaN,NaN,Never smoked,00000,Good,No,No,No,No,No,No,Yes
1,Alabama,2024000002,No,NaN,NaN,"Urban counties (_URBNRRL = 1,2,3,4,5)",Age 80 or older,Male,"White only, non-Hispanic",College 4 years or more (College graduate),"$200,000 or more","Yes, only one",No,81.65,1.78,25.83,Yes,NaN,NaN,Former smoker,00000,Excellent,No,Yes,No,No,No,No,Yes
2,Alabama,2024000003,No,NaN,NaN,"Urban counties (_URBNRRL = 1,2,3,4,5)",Age 55 to 59,Male,"White only, non-Hispanic",College 1 year to 3 years (Some college or tec...,Refused,No,Yes,88.45,1.98,22.53,Yes,NaN,NaN,Current smoker - now smokes every day,01400,Very good,No,No,No,No,No,No,Yes
3,Alabama,2024000004,No,NaN,NaN,"Urban counties (_URBNRRL = 1,2,3,4,5)",Age 80 or older,Male,"White only, non-Hispanic",College 4 years or more (College graduate),"Less than $50,000 ($35,000 to < $50,000)","Yes, only one",No,74.84,1.73,25.09,Yes,NaN,NaN,Never smoked,00000,Excellent,No,No,No,No,No,No,Yes
4,Alabama,2024000005,No,NaN,NaN,"Urban counties (_URBNRRL = 1,2,3,4,5)",Age 45 to 49,Male,"White only, non-Hispanic",College 1 year to 3 years (Some college or tec...,"Less than $20,000 ($15,000 to < $20,000)","Yes, only one",No,58.97,1.73,19.77,No,NaN,NaN,Never smoked,00000,Good,No,No,No,No,No,No,No



Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457670 entries, 0 to 457669
Data columns (total 29 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   _STATE    457670 non-null  object 
 1   SEQNO     457670 non-null  object 
 2   DIABETE4  457666 non-null  object 
 3   PREDIAB2  159195 non-null  object 
 4   DIABTYPE  13818 non-null   object 
 5   _URBSTAT  443047 non-null  object 
 6   _AGEG5YR  457670 non-null  object 
 7   SEXVAR    457670 non-null  object 
 8   _RACE     457670 non-null  object 
 9   EDUCA     457663 non-null  object 
 10  INCOME3   448401 non-null  object 
 11  PERSDOC3  457667 non-null  object 
 12  MEDCOST1  457664 non-null  object 
 13  WTKG3     421278 non-null  float64
 14  HTM4      433610 non-null  float64
 15  _BMI5     414633 non-null  float64
 16  EXERANY2  457667 non-null  object 
 17  SSBSUGR2  115602 non-null  object 
 18  SSBFRUT3  115316 non-null  object 
 19  _SMOKER3  457670 non-null  ob

### Missing Value Analysis
Check for missing values across all columns to identify data completeness issues.

In [27]:
# Calculate missing values and percentages
missing_df = pd.DataFrame({
    'Column': transform_df.columns,
    'Missing_Count': transform_df.isnull().sum(),
    'Missing_Percent': (transform_df.isnull().sum() / len(transform_df) * 100).round(2)
})

# Sort by missing count descending
missing_df = missing_df.sort_values('Missing_Count', ascending=False)

print("Missing Value Summary:")
print(f"Total rows: {len(transform_df):,}\n")
print(missing_df.to_string(index=False))

# Highlight columns with significant missing data (>5%)
high_missing = missing_df[missing_df['Missing_Percent'] > 5]
if len(high_missing) > 0:
    print(f"\n⚠️  Columns with >5% missing data:")
    print(high_missing.to_string(index=False))
else:
    print("\n✓ No columns have >5% missing data")

Missing Value Summary:
Total rows: 457,670

  Column  Missing_Count  Missing_Percent
DIABTYPE         443852            96.98
SSBFRUT3         342354            74.80
SSBSUGR2         342068            74.74
PREDIAB2         298475            65.22
   _BMI5          43037             9.40
   WTKG3          36392             7.95
    HTM4          24060             5.26
_URBSTAT          14623             3.20
 INCOME3           9269             2.03
   EDUCA              7             0.00
MEDCOST1              6             0.00
CHCKDNY2              6             0.00
 GENHLTH              5             0.00
HAVARTH4              5             0.00
ADDEPEV3              5             0.00
DIABETE4              4             0.00
CVDCRHD4              3             0.00
 ASTHMA3              3             0.00
PERSDOC3              3             0.00
EXERANY2              3             0.00
CVDSTRK3              3             0.00
CVDINFR4              2             0.00
  SEXVAR     

### Data Type Verification
Verify that columns have the expected data types after parsing and value label mapping.

In [28]:
# Define expected types for different column categories
categorical_cols = ['_STATE', 'DIABETE4', 'PREDIAB2', 'DIABTYPE', '_URBSTAT', '_AGEG5YR', 
                    'SEXVAR', '_RACE', 'EDUCA', 'INCOME3', 'PERSDOC3', 'MEDCOST1', 
                    'EXERANY2', '_SMOKER3', 'GENHLTH', 'CVDINFR4', 'CVDCRHD4', 
                    'CVDSTRK3', 'CHCKDNY2', 'ASTHMA3', 'ADDEPEV3', 'HAVARTH4']

numeric_cols = ['SEQNO', 'WTKG3', 'HTM4', '_BMI5', 'SSBSUGR2', 'SSBFRUT3', '_DRNKWK3']

print("Data Type Summary:\n")
print(f"{'Column':<15} {'Current Type':<15} {'Expected Category':<20}")
print("=" * 50)

for col in transform_df.columns:
    current_type = str(transform_df[col].dtype)
    if col in categorical_cols:
        expected = "Categorical/String"
    elif col in numeric_cols:
        expected = "Numeric"
    else:
        expected = "Unknown"
    
    print(f"{col:<15} {current_type:<15} {expected:<20}")

print(f"\n✓ Data type summary complete")
print(f"  Categorical columns: {len(categorical_cols)} (human-readable labels)")
print(f"  Numeric columns: {len(numeric_cols)} (transformed with proper decimals)")

Data Type Summary:

Column          Current Type    Expected Category   
_STATE          object          Categorical/String  
SEQNO           object          Numeric             
DIABETE4        object          Categorical/String  
PREDIAB2        object          Categorical/String  
DIABTYPE        object          Categorical/String  
_URBSTAT        object          Categorical/String  
_AGEG5YR        object          Categorical/String  
SEXVAR          object          Categorical/String  
_RACE           object          Categorical/String  
EDUCA           object          Categorical/String  
INCOME3         object          Categorical/String  
PERSDOC3        object          Categorical/String  
MEDCOST1        object          Categorical/String  
WTKG3           float64         Numeric             
HTM4            float64         Numeric             
_BMI5           float64         Numeric             
EXERANY2        object          Categorical/String  
SSBSUGR2        object    

### Range Checks for Numeric Variables
Verify that numeric columns have reasonable values within expected ranges.

In [29]:
# Define numeric columns to check
numeric_check_cols = {
    'WTKG3': {'name': 'Weight (kg)', 'min': 23.00, 'max': 295.00},
    'HTM4': {'name': 'Height (m)', 'min': 0.91, 'max': 2.44},
    '_BMI5': {'name': 'BMI', 'min': 0.01, 'max': 99.99},
    'SSBSUGR2': {'name': 'Sugar soda freq', 'min': 0, 'max': 999},
    'SSBFRUT3': {'name': 'Sugar drink freq', 'min': 0, 'max': 999},
    '_DRNKWK3': {'name': 'Drinks/week', 'min': 0, 'max': 99900}
}

print("Numeric Variable Range Summary:\n")
print(f"{'Variable':<15} {'Description':<20} {'Count':<10} {'Min':<10} {'Max':<10} {'Mean':<10} {'Valid Range':<20}")
print("=" * 105)

for col, info in numeric_check_cols.items():
    if col in transform_df.columns:
        # Convert to numeric, coercing errors to NaN
        numeric_vals = pd.to_numeric(transform_df[col], errors='coerce')
        
        count = numeric_vals.notna().sum()
        min_val = numeric_vals.min() if count > 0 else None
        max_val = numeric_vals.max() if count > 0 else None
        mean_val = numeric_vals.mean() if count > 0 else None
        
        # Format values
        min_str = f"{min_val:.2f}" if min_val is not None else "N/A"
        max_str = f"{max_val:.2f}" if max_val is not None else "N/A"
        mean_str = f"{mean_val:.2f}" if mean_val is not None else "N/A"
        range_str = f"{info['min']}-{info['max']}"
        
        print(f"{col:<15} {info['name']:<20} {count:<10,} {min_str:<10} {max_str:<10} {mean_str:<10} {range_str:<20}")
        
        # Check for out-of-range values
        if min_val is not None and max_val is not None:
            out_of_range = ((numeric_vals < info['min']) | (numeric_vals > info['max'])).sum()
            if out_of_range > 0:
                print(f"  ⚠️  {out_of_range:,} values outside expected range")

print("\n✓ Range check complete")

Numeric Variable Range Summary:

Variable        Description          Count      Min        Max        Mean       Valid Range         
WTKG3           Weight (kg)          421,278    22.68      290.30     83.00      23.0-295.0          
  ⚠️  14 values outside expected range
HTM4            Height (m)           433,610    0.91       2.41       1.70       0.91-2.44           
_BMI5           BMI                  414,633    12.00      99.84      28.56      0.01-99.99          
SSBSUGR2        Sugar soda freq      115,602    101.00     999.00     532.12     0-999               
SSBFRUT3        Sugar drink freq     115,316    101.00     999.00     599.14     0-999               
_DRNKWK3        Drinks/week          457,670    0.00       99900.00   10443.93   0-99900             

✓ Range check complete


## Data Removal
Remove records with data quality issues identified during validation.

### Remove Out-of-Range Numeric Records
Remove records where any numeric variable is outside its expected range as specified in the data dictionary.

In [14]:
# Create a copy of transform_df for cleaning
# This allows Data Removal section to be rerun without corrupting transform_df
clean_df = transform_df.copy()

# Count rows before removal
rows_before = len(clean_df)

# Track which records to remove (combine all out-of-range conditions)
records_to_remove = pd.Series([False] * len(clean_df), index=clean_df.index)

print("Checking numeric variables for out-of-range values:\n")

for col, info in numeric_check_cols.items():
    if col in clean_df.columns:
        # Convert to numeric, coercing errors to NaN
        numeric_vals = pd.to_numeric(clean_df[col], errors='coerce')
        
        # Identify out-of-range values (excluding NaN)
        out_of_range = ((numeric_vals < info['min']) | (numeric_vals > info['max'])) & numeric_vals.notna()
        out_of_range_count = out_of_range.sum()
        
        if out_of_range_count > 0:
            print(f"  {col} ({info['name']}): {out_of_range_count:,} out-of-range values")
            print(f"    Range: {info['min']}-{info['max']}")
            
            # Add to removal mask
            records_to_remove = records_to_remove | out_of_range
        else:
            print(f"  {col} ({info['name']}): ✓ All values in range")

# Count total unique records to remove
total_to_remove = records_to_remove.sum()

print(f"\nTotal unique records to remove: {total_to_remove:,}")

if total_to_remove > 0:
    # Show sample of records being removed
    print(f"\nSample of records being removed (first 10):")
    sample_cols = ['_STATE', 'SEQNO'] + [col for col in numeric_check_cols.keys() if col in clean_df.columns]
    print(clean_df[records_to_remove][sample_cols].head(10).to_string(index=False))
    
    # Remove the records
    clean_df = clean_df[~records_to_remove].copy()
    
    rows_after = len(clean_df)
    rows_removed = rows_before - rows_after
    
    print(f"\n✓ Removed {rows_removed:,} records ({rows_removed/rows_before*100:.3f}%)")
    print(f"  Rows before: {rows_before:,}")
    print(f"  Rows after: {rows_after:,}")
else:
    print(f"\n✓ No out-of-range values found in any numeric column")

Checking numeric variables for out-of-range values:

  WTKG3 (Weight (kg)): 14 out-of-range values
    Range: 23.0-295.0
  HTM4 (Height (m)): ✓ All values in range
  _BMI5 (BMI): ✓ All values in range
  SSBSUGR2 (Sugar soda freq): ✓ All values in range
  SSBFRUT3 (Sugar drink freq): ✓ All values in range
  _DRNKWK3 (Drinks/week): ✓ All values in range

Total unique records to remove: 14

Sample of records being removed (first 10):
       _STATE      SEQNO  WTKG3  HTM4  _BMI5 SSBSUGR2 SSBFRUT3 _DRNKWK3
   California 2024007971  22.68  1.65    NaN      NaN      NaN    00000
     Colorado 2024004132  22.68   NaN    NaN      NaN      NaN    00000
     Colorado 2024006745  22.68  1.40    NaN      NaN      NaN    00000
     Colorado 2024009505  22.68  1.50    NaN      NaN      NaN    00000
      Florida 2024008581  22.68   NaN    NaN      NaN      NaN    00000
       Hawaii 2024001165  22.68  1.63    NaN      NaN      NaN    00000
     Illinois 2024006574  22.68  1.52    NaN      NaN      Na

## Write Data
Save the cleaned and transformed dataset to file for use in EDA and modeling.

In [15]:
if output_file:
    # Determine file format from extension
    if output_file.endswith('.csv'):
        clean_df.to_csv(output_file, index=False)
        print(f"✓ Saved cleaned dataset to CSV: {output_file}")
    elif output_file.endswith('.parquet'):
        clean_df.to_parquet(output_file, index=False)
        print(f"✓ Saved cleaned dataset to Parquet: {output_file}")
    elif output_file.endswith('.pkl') or output_file.endswith('.pickle'):
        clean_df.to_pickle(output_file)
        print(f"✓ Saved cleaned dataset to Pickle: {output_file}")
    else:
        print(f"⚠️  Unsupported file format: {output_file}")
        print(f"   Supported formats: .csv, .parquet, .pkl, .pickle")
    
    # Print summary
    print(f"\nDataset summary:")
    print(f"  Rows: {len(clean_df):,}")
    print(f"  Columns: {len(clean_df.columns)}")
    print(f"  Memory usage: {clean_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
else:
    print("⚠️  No output file specified in parameters section")
    print("   Set 'output_file' variable to save the cleaned dataset")
    print(f"\nCleaned dataset available as 'clean_df':")
    print(f"  Rows: {len(clean_df):,}")
    print(f"  Columns: {len(clean_df.columns)}")

✓ Saved cleaned dataset to Pickle: diabetes_data.pickle

Dataset summary:
  Rows: 457,656
  Columns: 29
  Memory usage: 648.46 MB
